In [1]:
#from unstructured_client import UnstructuredClient
from unstructured_client.models import shared
#from unstructured_client.models.errors import SDKError

import chromadb
import os

from unstructured.chunking.title import chunk_by_title
from unstructured.partition.md import partition_md
from unstructured.partition.pptx import partition_pptx
from unstructured.staging.base import dict_to_elements

import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass()

folder="docs/"
for filename in os.listdir(folder):
     if filename.endswith(".pdf"):
        print(filename)
        with open(folder+filename, "rb") as f:
            files=shared.Files(
            content=f.read(),
            file_name=filename)

In [2]:
#print(files)

req = shared.PartitionParameters(
    files=files,
    strategy="hi_res",
    hi_res_model_name="yolox",
    pdf_infer_table_structure=True,
    skip_infer_table_types=[],
)

In [3]:
#print(req)

try:
    resp = s.general.partition(req)
    pdf_elements = dict_to_elements(resp.elements)
except SDKError as e:
    print(e)

In [4]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("docs/")
documents=loader.load()

In [5]:
#print(docs)

In [6]:
# import
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter

# load the document and split it into chunks
#loader = TextLoader("../../modules/state_of_the_union.txt")
#documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)


In [7]:
print(docs)

[Document(page_content='Arnab Basak  \nKolkata, West Bengal, India  \n+91 916398973 5 \narnabbasak 99@gmail.com  \n \nSummary  \nPassionate and dedicated advocate for Data Science and Artificial Intelligence.  Dedicated to leveraging \ncutting -edge technology to solve real -world challenges. Eager to contribute to impactful projects and \ncontinuously expand my expertise in the dynamic field of Data Science and AI .  \nEducation  \n❖ 2022 -Present:  Pursuing M.TECH in Computer Science and Engineering with specialization in Data \nScience and Engineering  from NIT Agartala . Have scored 8.60 CGPA in the first three semesters . \n❖ 2017 -2021 : Acquired B.TECH Degree in Computer Science and Engineering from Guru Nanak \nInstitute of Technology under MAKAUT with 8.36 DGPA.  \n❖ 2017 :  Passed Higher Secondary Examination from  St. Stephens  School under Council for Indian \nSchool Certificate Examination with 90.75%  \n❖ 2015 :  Passed Secondary Examination from St. Stephens School  unde

In [8]:

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

# query it
#query = "What did the president say about Ketanji Brown Jackson"
#docs = db.similarity_search(query)

# print results
#print(docs[0].page_content)

INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2


e:\GitHub\LangChain-Chatbot\lchain\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


INFO: Use pytorch device_name: cuda
INFO: Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


e:\GitHub\LangChain-Chatbot\lchain\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [9]:
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 6}
)

In [10]:
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import OpenAI
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

#OPENAI_API_KEY = getpass()

In [12]:
template = """You are an AI assistant for answering questions about the Donut document understanding model.
You are given the following extracted parts of a long document and a question. Provide a conversational answer.
If you don't know the answer, just say "Hmm, I'm not sure." Don't try to make up an answer.
If the question is not about Donut, politely inform them that you are tuned to only answer questions about Donut.
Question: {question}
=========
{context}
=========
Answer in Markdown:"""
prompt = PromptTemplate(template=template, input_variables=["question", "context"])

for key, value in os.environ.items():
    if key==OPENAI_API_KEY:   
     print('{}: {}'.format(key, value))

In [13]:
import os
# importing necessary functions from dotenv library
from dotenv import load_dotenv, dotenv_values 
# loading variables from .env file
load_dotenv() 
 
# accessing and printing value
#print(os.getenv("OPENAI_API_KEY"))
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [14]:
print(os.environ["OPENAI_API_KEY"])

sk-proj-HcKJbLSBBxa1IN5X7SP0T3BlbkFJOzw7AnN6N9u5lLMayQoK


In [15]:
#from dotenv import load_dotenv, find_dotenv
#from pathlib import Path
#load_dotenv(Path(".env"))
#import env
#os.environ[OPENAI_API_KEY] = 

In [16]:
llm = OpenAI(temperature=0)

doc_chain = load_qa_with_sources_chain(llm, chain_type="map_reduce")
question_generator_chain = LLMChain(llm=llm, prompt=prompt)
qa_chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator_chain,
    combine_docs_chain=doc_chain,
)

e:\GitHub\LangChain-Chatbot\lchain\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
e:\GitHub\LangChain-Chatbot\lchain\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ConversationalRetrievalChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use create_history_aware_retriever together with create_retrieval_chain (see example in docstring) instead.
  warn_deprecated(


In [17]:
#https://realpython.com/build-llm-rag-chatbot-with-langchain/

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [18]:
from langchain.embeddings import HuggingFaceEmbeddings

In [19]:
import transformers
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

#model_id = 'H:/llama2/llama2-7b-chat-hf/'
#model_config = transformers.AutoConfig.from_pretrained(
#    model_id,
#)

#bnb_config = transformers.BitsAndBytesConfig(
#    load_in_4bit=True,
#    bnb_4bit_quant_type='nf4',
#    bnb_4bit_use_double_quant=True,
#    bnb_4bit_compute_dtype=bfloat16
#)

In [20]:
torch.cuda.is_available()

True

In [21]:
torch.cuda.device_count()

1

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    trust_remote_code=True,
    max_new_tokens=1024
)
print(device)


model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)

In [22]:
from groq import Groq
from langchain_groq.chat_models import ChatGroq

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
llm = ChatGroq(model="Llama3-70b-8192", temperature=0)

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)


In [23]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [24]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [25]:
conversational_rag_chain.invoke(
    {"input": "What is the document about?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

INFO: HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


'This document appears to be a resume or curriculum vitae of Arnab Basak, highlighting his education, experience, skills, and achievements in the field of Data Science and Artificial Intelligence.'